<a href="https://colab.research.google.com/github/hamzamusht550/Amazon-Clone-Frontend/blob/main/MKV_Combine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

#@title Install Dependencies

from IPython.display import clear_output
from google.colab import drive
from google.colab import auth

!apt-get install -y mediainfo ffmpeg mkvtoolnix aria2
!pip install patool pymediainfo ffmpeg-python pyrogram tgcrypto nest_asyncio

drive.mount('/content/drive')
auth.authenticate_user()
clear_output()

In [ ]:

#@title Mediainfo MKV

MKV_INFO = "" #@param {type: "string"}

from IPython.display import clear_output
clear_output()

!ffprobe -i "{MKV_INFO}"
!mediainfo "{MKV_INFO}"
!du -h "{MKV_INFO}"
!du -BM "{MKV_INFO}"

In [ ]:

#@title MKV Merger
# green #00FF02

MKV_FILES = ""  #@param {type: "string"}
LANGUAGE = "All"  #@param ["All", "Custom", "1st_Track", "Hindi", "English", "Japanese", "Tamil", "Telugu", "Malayalam", "Hindi_English", "Hindi_Japanese", "English_Japanese"]
Custom_FFMPEG = "-map 0:v -map 0:a:m:language:hin -map 0:s? -c copy"  #@param {type: "string"}
OUTPUT_DIR = ""  #@param {type: "string"}
MAX_FileSize_MB = 0  #@param {type: "number"}
USE_Splitting = False  #@param {type: "boolean"}
Overwrite_OUTPUT = False  #@param {type: "boolean"}
JUST_DOWNLOAD = False  #@param {type: "boolean"}
#Delete_Extracted_DIR = False  #@param {type: "boolean"}
#@markdown ***Start 00:00:00 / End 00:00:10***
trim_start = ""  #@param {type: "string"}
trim_end = ""  #@param {type: "string"}


from IPython.display import clear_output
from IPython.display import HTML
import subprocess
import os
import re
import requests
import patoolib
from google.colab import drive
from zipfile import ZipFile
import ffmpeg
from pymediainfo import MediaInfo
import mimetypes
# Mount Google Drive
drive.mount('/content/drive')
clear_output()


def printColor(txt, color, htmlTag=''):
    if not htmlTag:
        print(f"\n\033[{color}m{txt}\033[0m")
        return

    html_output = f"""
    <{htmlTag} onclick="copyToClipboard('{txt}')" style='color:{color};'>{txt}
    </{htmlTag}>
    """

    html_output += """
    <script>
    function copyToClipboard(text) {
        navigator.clipboard.writeText(text).then(function() {
            alert('Copied to clipboard: ' + text);
        }, function(err) {
            alert('Failed to copy: ' + err);
        });
    }
    </script>
    """
    display(HTML(html_output))

def deleteFolder(input_folder):
    !rm -rf "{input_folder}"
    print(f"Folder Deleted: {input_folder}")

def strToList(listStr, separator):
    myList = [_.strip() for _ in listStr.split(separator)]
    myList = [_ for _ in myList if _ and _.strip()] # Remove empty items from list
    finalList = list(set(myList)) # Remove duplicates items from list
    showAllElements = "\n".join(f"{i+1} ==> {ele}" for i, ele in enumerate(finalList))
    print(showAllElements, "\n")
    return finalList

def extract_file_info(file_path):
    file_path = os.path.abspath(file_path)
    # parent_dir_path = os.path.dirname(file_path)
    # file_name = os.path.basename(file_path)
    parent_dir_path, file_name = os.path.split(file_path)
    _name, _ext = os.path.splitext(file_name)
    return parent_dir_path,  file_name, _name, _ext[1:]

def add_suffix_to_file(file_path, suffix="_Suffix"):
    file_path = os.path.abspath(file_path)
    parent_dir_path, file_name = os.path.split(file_path)
    name, ext = os.path.splitext(file_name)
    new_file_name = f"{name}{suffix}{ext}"
    new_file_path = os.path.join(parent_dir_path, new_file_name)
    return new_file_path

def getMKVFilesList(folder_path, relative=0):
    folder_path = os.path.abspath(folder_path)
    all_files = []
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            if file.endswith('.mkv'):
                full_path = os.path.join(root, file)
                if relative == 1:
                    full_path = os.path.relpath(full_path, folder_path)
                all_files.append(full_path)

    all_files.sort(key=str.lower)
    return all_files


def extractArchive(Archive_File, extracted_folder=''):
    Archive_File = os.path.abspath(Archive_File)
    if not extracted_folder:
        extracted_folder = extract_file_info(Archive_File)[2]
    if os.path.exists(extracted_folder):
        printColor(f"Archive File Already Extracted:\n{extracted_folder}\nPlease Delete it if you want to proceed", "#00FF02", "h3")
        return extracted_folder

    os.makedirs(extracted_folder, exist_ok=True)
    printColor(f"Extracting archive to: {extracted_folder}", 92)
    patoolib.extract_archive(Archive_File, outdir=extracted_folder)
    printColor(f"Extraction complete: {extracted_folder}", "#00FF02", "h3")
    return extracted_folder


def runCommand(cmd, showLog=0):
    if showLog == 1:
        # Print logs in real-time, if 2nd param is 1
        printColor(f"Running command: {cmd}", 92)
        process = subprocess.Popen(cmd, shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT)
        for line in iter(process.stdout.readline, b''):
            print(line.decode("utf-8"), end="")

        process.stdout.close()
        process.wait()

    else:
        process = subprocess.run(cmd, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        return process


def Aria2Downloader(url, output_dir=''):
    printColor(f"Downloading ==> {url}", 92)
    Aria2 = f"aria2c --continue=true --file-allocation=none -x 16 -s 16 -d \"{output_dir}\" \"{url}\""
    if url.startswith("magnet") or url.endswith(".torrent"):
        Aria2 = f"aria2c --bt-enable-lpd=true --bt-save-metadata=true --continue=true --file-allocation=none -x 16 -s 16 -d \"{output_dir}\" \"{url}\""
    printColor(f"Running command: {Aria2}", 92)

    output_file_path = ""
    process = subprocess.Popen(Aria2, shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT)
    for line in iter(process.stdout.readline, b''):
        txt = line.decode("utf-8")
        print(txt, end="")
        if txt.find("Download complete: ") > -1:
            output_file_path = txt.split("Download complete: ")
            output_file_path = output_file_path[1].split("\n")[0]
    process.stdout.close()
    process.wait()

    if output_file_path:
        output_file_path = os.path.abspath(output_file_path)
        printColor(f"\nDownload Success ==> {output_file_path}", 92)  # Green color
        return output_file_path
    else:
        printColor(f"Download Error Status code({process.returncode}) ==> {url}", 91)  # Red color
    return url


def getLanguage():
    switch_dict = {
        "All": "-map 0:v -map 0:a -map 0:s? -c copy",
        "Custom": Custom_FFMPEG,
        "1st_Track": "-c copy",
        "Hindi": "-map 0:v -map 0:a:m:language:hin -map 0:s? -c copy",
        "English": "-map 0:v -map 0:a:m:language:eng -map 0:s? -c copy",
        "Japanese": "-map 0:v -map 0:a:m:language:jpn -map 0:s? -c copy",
        "Tamil": "-map 0:v -map 0:a:m:language:jpn -tam 0:s? -c copy",
        "Telugu": "-map 0:v -map 0:a:m:language:jpn -tel 0:s? -c copy",
        "Malayalam": "-map 0:v -map 0:a:m:language:mal -map 0:s? -c copy",
        "Hindi_English": "-map 0:v -map 0:a:m:language:hin -map 0:a:m:language:eng -map 0:s? -c copy",
        "Hindi_Japanese": "-map 0:v -map 0:a:m:language:hin -map 0:a:m:language:jpn -map 0:s? -c copy",
        "English_Japanese": "-map 0:v -map 0:a:m:language:eng -map 0:a:m:language:jpn -map 0:s? -c copy"
    }
    return switch_dict.get(LANGUAGE, "Invalid_Lang")  # Default case


def create_concat_list(filepaths, output_file="list.txt"):
    def sanitize_name(name):
        # Characters to replace
        special_chars = ',:;"!\'?*'
        # Replace each special character with '_'
        for char in special_chars:
            name = name.replace(char, '_')
        return name

    def sanitize_path(path):
        # Split the path into components
        folders = []
        while True:
            path, folder = os.path.split(path)
            if folder:
                folders.append(folder)
            else:
                if path:
                    folders.append(path)
                break
        # Reverse to get the correct order (from root to leaf)
        folders.reverse()
        # Sanitize each folder name
        sanitized_folders = [sanitize_name(folder) for folder in folders]
        # Reconstruct the sanitized path
        sanitized_path = os.path.join(*sanitized_folders)
        return sanitized_path

    def rename_folders_and_files(path):
        # Sanitize the entire path
        sanitized_path = sanitize_path(path)
        # Rename folders and files if the sanitized path is different
        if sanitized_path != path:
            os.renames(path, sanitized_path)
        return sanitized_path

    with open(output_file, "w") as list_file:
        for filepath in filepaths:
            filepath = os.path.abspath(filepath)
            # Sanitize the entire path (including folders)
            sanitized_filepath = rename_folders_and_files(filepath)
            # Write the sanitized filepath to list.txt
            list_file.write(f"file '{sanitized_filepath}'\n")
    return output_file


def getFFmpegOptions():
    FFMPEG_OPTIONS = ""
    FFMPEG_OPTIONS += f" {getLanguage()} -map_metadata -1 -metadata title=\"@Anime_Closx\" -ignore_unknown"
    FFMPEG_OPTIONS += " -y" if Overwrite_OUTPUT else ""
    return FFMPEG_OPTIONS


def mergeMKVs(video_paths, output_dir):
    if JUST_DOWNLOAD:
        video_paths_Str = '\n'.join(video_paths)
        printColor("You have enabled JUST_DOWNLOAD", "#00FF02", "h3")
        printColor(f"Videos to Merge:\n{video_paths_Str}", "#00FF02", "h3")
        return
    video_paths.sort(key=str.lower)
    if len(video_paths) < 2:
        print(video_paths, f" ==> Can't Merge {len(video_paths)} video")
        return video_paths

    os.makedirs(output_dir, exist_ok=True)
    base_name = extract_file_info(video_paths[0])[2]
    base_output_path = os.path.join(output_dir, base_name)

    # Create list.txt in the output directory
    list_path = os.path.join(output_dir, f"{base_name}.txt")
    create_concat_list(video_paths, list_path)

    command = f"ffmpeg -f concat -safe 0 -i \"{list_path}\" {getFFmpegOptions()} \"{base_output_path}_merged.mkv\""
    if MAX_FileSize_MB > 999 and USE_Splitting:
        command = f"ffmpeg -f concat -safe 0 -i \"{list_path}\" {getFFmpegOptions()} temp.mkv"
        runCommand(command, 1)
        command = f"mkvmerge -o \"{base_output_path}_%03d.mkv\" --split size:{MAX_FileSize_MB}M temp.mkv"
        runCommand(command, 1)

    runCommand(command, 1)
    return base_output_path


def singleMKV(video, output_video):
    if JUST_DOWNLOAD:
        printColor("You have enabled JUST_DOWNLOAD", "#00FF02", "h3")
        printColor(f"Single Video ==> {video}", "#00FF02", "h3")
        return
    video = os.path.abspath(video)
    if not (output_video and output_video.endswith('.mkv')):
        output_video = add_suffix_to_file(video, f" ({LANGUAGE})")
    output_video = os.path.abspath(output_video)

    trim = f"-ss {trim_start} -to {trim_end}" if trim_start else ""
    command = f"ffmpeg -i \"{video}\" {trim} {getFFmpegOptions()} \"{output_video}\""
    runCommand(command, 1)
    printColor(f"Complete Single Video ==> {output_video}", "#00FF02", "h3")
    return output_video


def main(filesListStr, output_dir):
    if not output_dir:
        output_dir = '/content/drive/MyDrive'
    valid_Files_List = []

    if os.path.exists(filesListStr):
        printColor(f"MKV_FILES str Exists ==> {filesListStr}", "#00FF02", "h3")
        if patoolib.is_archive(filesListStr): # Type Archive
            _Folder = extractArchive(filesListStr)
            mergeMKVs(getMKVFilesList(_Folder), output_dir)
        elif os.path.isdir(filesListStr): # Type Folder
            mergeMKVs(getMKVFilesList(filesListStr), output_dir)
        elif filesListStr.endswith('.mkv'): # Type MKV
            print(f"Single MKV ==> {filesListStr}")
            singleMKV(filesListStr, output_dir)
        return

    Files_List = strToList(filesListStr, "|")
    _File = Files_List[0]

    if len(Files_List) == 0:
        printColor(Files_List, " ==> Give At Least 2 files for Merging", "red", "h3")
        return
    elif len(Files_List) == 1:
        _File = Files_List[0]
        if _File.startswith(('http', 'magnet:') or _File.endswith('.torrent')):
            _File = Aria2Downloader(_File)
            if os.path.exists(_File):
                if patoolib.is_archive(_File):
                    _Folder = extractArchive(_File)
                    mergeMKVs(getMKVFilesList(_Folder), output_dir)
                elif _File.endswith('.mkv'):
                    printColor(f"Single MKV ==> {_File}", "#00FF02", "h3")
                    singleMKV(_File, output_dir)
        return

    if len(Files_List) > 1:
        for i, valid_File in enumerate(Files_List):
            if valid_File.startswith(('http', 'magnet:') or valid_File.endswith('.torrent')):
                valid_File = Aria2Downloader(valid_File, "download_" + extract_file_info(Files_List[0])[2] )
            if os.path.exists(valid_File):
                if valid_File.endswith('.mkv'):
                    valid_File = os.path.abspath(valid_File)
                    valid_Files_List.append(valid_File)
                else:
                    printColor(f"File Exists but Unsupported File Format: {valid_File}", "red", "h3")
            else:
                printColor(f"Error FILES {i+1}: {valid_File}", "red", "h3")
                return
        valid_Files_Str = '\n'.join(valid_Files_List)
        printColor(f"Valid Files are:\n{valid_Files_Str}", 92)

    if len(valid_Files_List) > 1:
        mergeMKVs(valid_Files_List, output_dir)

if MAX_FileSize_MB <= 999 and USE_Splitting:
    printColor(f"Split Size can't be Smaller than 1000MB", "red", "h2")
else:
    main(MKV_FILES, OUTPUT_DIR)

In [ ]:

#@title URL Downloader

URLs = "https://tg-movies4u.tgmkvofcl.workers.dev/230570?hash=AgADRR"  # @param {type: "string"}
OUTPUT_DIR = "/content"  #@param {type: "string"}
Select_Downloader = "wget"  #@param ["Aria2", "wget", "Aria2os"]

import subprocess

def strToList(listStr, separator=","):
    listStr = ' '.join(listStr.split()) # Remove extra spaces from String
    myList = [_.strip() for _ in listStr.split(separator)]
    myList = [_ for _ in myList if _ and _.strip()] # Remove empty items from list
    finalList = list(set(myList)) # Remove duplicates items from list
    showAllElements = "\n".join(f"{i+1} ==> {ele}" for i, ele in enumerate(finalList))
    print(showAllElements, "\n\n")
    return finalList


def main(urlStr):
    URLs_List = strToList(urlStr, ",")
    for i, _url in enumerate(URLs_List):
        print(f"\n{i+1} ==> {_url}")
        match Select_Downloader:
            case "Aria2":
                !aria2c --continue=true --file-allocation=none -x 16 -s 16 -d "{OUTPUT_DIR}" "{_url}"
            case "wget":
                !wget -P "{OUTPUT_DIR}" "{_url}"
            case "Aria2os":
                cmd = f"aria2c --continue=true --file-allocation=none -x 16 -s 16 -d \"{OUTPUT_DIR}\" \"{_url}\""
                process = subprocess.run(cmd, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
                file_path = process.stdout.decode('utf-8')
                file_path = file_path.split("Download complete: ")
                file_path = file_path[1].split("\n")[0]
                print(os.path.exists(file_path))
                print(file_path)


main(URLs)

In [ ]:

#@title Show MKV tracks

FILES_Folder = "/content/Reborn"  #@param {type: "string"}
Output_HTML = True  #@param {type: "boolean"}

from IPython.display import HTML
from pymediainfo import MediaInfo
import os

def get_track_info(file_path):
    try:
        media_info = MediaInfo.parse(file_path)
        tracks_info = []
        for track in media_info.tracks:
            if track.track_type in ["Video", "Audio", "Text"]:  # Only show relevant tracks
                track_id = track.track_id or "Unknown"
                track_type = track.track_type or "Unknown"
                codec = track.codec_id or "Unknown"
                language = track.language or "Unknown"
                title = track.title or "Untitled"
                tracks_info.append((track_id, track_type, codec, language, title))
        return tracks_info
    except Exception as e:
        print(f"Error occurred:\n{e}")
        return []

def getMKVFilesList(folder_path, relative=0):
    all_files = []
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            if file.endswith('.mkv'):
                full_path = os.path.join(root, file)
                if relative == 1:
                    full_path = os.path.relpath(full_path, folder_path)
                all_files.append(full_path)

    all_files.sort(key=str.lower)
    return all_files

def compare_tracks(tracks1, tracks2):
    return tracks1 == tracks2

def main(folder_path, use_html=True):
    files = getMKVFilesList(folder_path)
    if not files:
        print("No MKV files found in the specified folder.")
        return

    reference_tracks = get_track_info(files[0])
    # HTML output with copy-to-clipboard buttons
    html_output = f"""
    <h3 onclick="copyToClipboard('{files[0]}')">Reference file: <b>{files[0]}</b>
    </h3>
    <ul>
    """
    for track in reference_tracks:
        html_output += f"<li>ID: {track[0]} | Type: {track[1]} | Codec: {track[2]} | Language: {track[3]} | Title: {track[4]}</li>"
        html_output += f"<li><b>Complete INFO: {track}</b></li>"
    html_output += "</ul>"

    for i, _File in enumerate(files[1:], start=1):
        current_tracks = get_track_info(_File)
        if reference_tracks != current_tracks:
            html_output += f"""
            <h3 onclick="copyToClipboard('{_File}')" style='color:red;'>
            {i+1} ==> {_File} (Different Tracks)
            </h3>
            """
        if reference_tracks != current_tracks:
         html_output += "<ul>"
         for track in current_tracks:
             html_output += f"<li>ID: {track[0]} | Type: {track[1]} | Codec: {track[2]} | Language: {track[3]} | Title: {track[4]}</li>"
             html_output += f"<li><b>Complete INFO: {track}</b></li>"
         html_output += "</ul>"

    # Add JavaScript for copying to clipboard
    html_output += """
    <script>
    function copyToClipboard(text) {
        navigator.clipboard.writeText(text).then(function() {
            alert('Copied to clipboard: ' + text);
        }, function(err) {
            alert('Failed to copy: ' + err);
        });
    }
    </script>
    """

    display(HTML(html_output))


main(FILES_Folder, Output_HTML)

In [ ]:
#@title File Delete

Delete_File = "/content/Party.Till.I.Die.S01.720p.AMZN.WEB-DL.DDP2.0.H.265-PrimeFix"  #@param {type: "string"}
!ls -lh "{Delete_File}"
!rm -rf "{Delete_File}"

In [ ]:

#@title Save Public Google Drive Files

from google.colab import drive
from google.colab import auth
from oauth2client.client import GoogleCredentials

from pydrive2.auth import GoogleAuth
from pydrive2.drive import GoogleDrive

from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

# Authenticate and initialize
auth.authenticate_user()

# Get credentials
credentials = GoogleCredentials.get_application_default()
service = build('drive', 'v3', credentials=credentials)

public_drive_urls = "https://drive.google.com/drive/folders/1-9dwm1Jh5giHjQq5eCdC2X2k1UaQ-h8D"  # @param {type: "string"}
# https://drive.google.com/file/d/13Qdkvg8uKGuZv2_8Z1kyUzCMSSlblEEb/view?usp=drivesdk
# https://drive.google.com/drive/folders/1NrBvFvL-Jro3VQBSQhHiOj4-oEEnCIUl
# https://drive.google.com/drive/folders/1glEPV-ISGauk8_HXQ6dV6AfH1HJRsVc6?usp=sharing


def strToList(listStr, separator=","):
    listStr = ' '.join(listStr.split()) # Remove extra spaces from String
    myList = [_.strip() for _ in listStr.split(separator)]
    myList = [_ for _ in myList if _ and _.strip()] # Remove empty items from list
    showAllURLs = "\n".join(f"{i+1}. {ele}" for i, ele in enumerate(myList))
    print(showAllURLs)
    return myList


def cleanURL(url):
    if not url:
        return url
    if "." in url:
        if not url.lower().startswith("http"):
            url = "http://" + url

    return url


def extractPublicDriveID(url):
    gauth = GoogleAuth()
    gauth.credentials = GoogleCredentials.get_application_default()
    drive = GoogleDrive(gauth)
    try:
        # Check if the URL is a file or folder and extract the ID accordingly
        file_id = ""
        if "file/d/" in url:
            file_id = url.split('/d/')[1].split('/')[0]
        elif "folders/" in url:
            file_id = url.split('folders/')[1].split('/')[0]
        else:
            return ""  # Invalid URL format

        if "?usp=sharing" in file_id:
            file_id = file_id.replace("?usp=sharing","")

        # Create a file object and load metadata using the extracted ID
        file = drive.CreateFile({'id': file_id})
        file.FetchMetadata()  # Fetch metadata to verify ID and access permissions
        return file['id']
    except Exception as e:
        return ""

def getDriveMetadata(_id, metadata_key):
    _metadata = service.files().get(fileId=_id, fields=metadata_key).execute()
    metadata_value = _metadata.get(metadata_key)
    return metadata_value

# Function to create a folder in your Drive
def create_drive_folder(name, parent_id=None):
    folder_metadata = {
        'name': name,
        'mimeType': 'application/vnd.google-apps.folder',
        'parents': [parent_id] if parent_id else ["root"]
    }
    folder = service.files().create(body=folder_metadata, fields='id').execute()
    return folder.get('id')

# Function to copy a drive file
def transfer_drive_file(file_id, name="", parent_id=None):
    if not name:
        name = getDriveMetadata(file_id, 'name')
    copied_file_metadata = {
        'name': name,
        'parents': [parent_id] if parent_id else ["root"]
    }
    copied_file = service.files().copy(fileId=file_id, body=copied_file_metadata).execute()
    # return name, copied_file.get('id')
    return name

# Function to recursively copy a folder
def transfer_drive_folder(folder_id, parent_id=None):
    # Get the folder metadata
    folder_name = getDriveMetadata(folder_id, 'name')

    # Create the folder in your Drive
    new_folder_id = create_drive_folder(folder_name, parent_id)

    # List all items in the folder
    results = service.files().list(
        q=f"'{folder_id}' in parents",
        fields="files(id, name, mimeType)"
    ).execute()

    items = results.get('files', [])

    for item in items:
        if item['mimeType'] == 'application/vnd.google-apps.folder':
            # Recursively copy subfolder
            transfer_drive_folder(item['id'], new_folder_id)
        else:
            # Copy individual file
            transfer_drive_file(item['id'], item['name'], new_folder_id)

    print(f"Copied folder '{folder_name}' to your Drive (ID: {new_folder_id})")


def downloadFromPublicDrive(Drive_URLs):
    colab_root = "/content/"
    drive_root = "/content/drive/MyDrive/"
    file_folder_name = ""
    file_folder_path = ""

    Drive_URLs_List = strToList(Drive_URLs)
    for url in Drive_URLs_List:
        public_drive_id = extractPublicDriveID(url)
        if "file/d/" in url:
            # Start copying the folder
            try:
                file_folder_name = transfer_drive_file(public_drive_id)
                file_folder_path = drive_root + file_folder_name
            except HttpError as error:
                print(f"Drive error occurred: {error}")
        elif "folders/" in url:
            # Start copying the folder
            try:
                transfer_drive_folder(public_drive_id)
                file_folder_name = getDriveMetadata(public_drive_id, 'name')
                file_folder_path = drive_root + file_folder_name
            except HttpError as error:
                print(f"Drive error occurred: {error}")

        print(f"Drive Transfer Successfull: {file_folder_name}")


downloadFromPublicDrive(public_drive_urls)


# _. _. _.  _. _. _.  _. _. _.  _. _. _.  _. _. _.  _. _. _.